# Question 1

In [4]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/teams/v1/2/players"

headers = {
	"x-rapidapi-key": "5d00263db8msh6b461e04fff0e6bp1aa84cjsn57ff489cc9c5",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())


{'player': [{'name': 'BATSMEN', 'imageId': 174146}, {'id': '11808', 'name': 'Shubman Gill', 'imageId': 616515, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '13940', 'name': 'Yashasvi Jaiswal', 'imageId': 591942, 'battingStyle': 'Left-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '13866', 'name': 'Sai Sudharsan', 'imageId': 717782, 'battingStyle': 'Left-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '576', 'name': 'Rohit Sharma', 'imageId': 616514, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '1413', 'name': 'Virat Kohli', 'imageId': 616517, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm medium'}, {'id': '7915', 'name': 'Suryakumar Yadav', 'imageId': 846028, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '9428', 'name': 'Shreyas Iyer', 'imageId': 616518, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '10896', 'name': '

In [5]:
players_data = response.json()
print(players_data)

{'player': [{'name': 'BATSMEN', 'imageId': 174146}, {'id': '11808', 'name': 'Shubman Gill', 'imageId': 616515, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '13940', 'name': 'Yashasvi Jaiswal', 'imageId': 591942, 'battingStyle': 'Left-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '13866', 'name': 'Sai Sudharsan', 'imageId': 717782, 'battingStyle': 'Left-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '576', 'name': 'Rohit Sharma', 'imageId': 616514, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '1413', 'name': 'Virat Kohli', 'imageId': 616517, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm medium'}, {'id': '7915', 'name': 'Suryakumar Yadav', 'imageId': 846028, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}, {'id': '9428', 'name': 'Shreyas Iyer', 'imageId': 616518, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm legbreak'}, {'id': '10896', 'name': '

In [6]:
players_data
players_data['player']

[{'name': 'BATSMEN', 'imageId': 174146},
 {'id': '11808',
  'name': 'Shubman Gill',
  'imageId': 616515,
  'battingStyle': 'Right-hand bat',
  'bowlingStyle': 'Right-arm offbreak'},
 {'id': '13940',
  'name': 'Yashasvi Jaiswal',
  'imageId': 591942,
  'battingStyle': 'Left-hand bat',
  'bowlingStyle': 'Right-arm legbreak'},
 {'id': '13866',
  'name': 'Sai Sudharsan',
  'imageId': 717782,
  'battingStyle': 'Left-hand bat',
  'bowlingStyle': 'Right-arm legbreak'},
 {'id': '576',
  'name': 'Rohit Sharma',
  'imageId': 616514,
  'battingStyle': 'Right-hand bat',
  'bowlingStyle': 'Right-arm offbreak'},
 {'id': '1413',
  'name': 'Virat Kohli',
  'imageId': 616517,
  'battingStyle': 'Right-hand bat',
  'bowlingStyle': 'Right-arm medium'},
 {'id': '7915',
  'name': 'Suryakumar Yadav',
  'imageId': 846028,
  'battingStyle': 'Right-hand bat',
  'bowlingStyle': 'Right-arm offbreak'},
 {'id': '9428',
  'name': 'Shreyas Iyer',
  'imageId': 616518,
  'battingStyle': 'Right-hand bat',
  'bowlingStyl

In [7]:
indian_players = []
current_role = None
for player in players_data['player']:    
    if 'id' not in player:
        current_role = player['name']
        continue
    role = player.get('role') or player.get('playingRole') or current_role
    indian_players.append({
        'id': player['id'],
        'name': player['name'],
        'role': role,
        'batting_style': player.get('battingStyle'),
        'bowling_style': player.get('bowlingStyle')
    })

In [8]:
import pandas as pd
indian_players_df = pd.DataFrame(indian_players)

In [38]:
import psycopg2
import requests
DB_CONFIG = {
    'host': 'localhost',
    'database': 'guviproj',
    'user': 'postgres',
    'password': 'ssbv7792',
    'port': 5432
    }   

In [55]:
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

Connected to the database successfully


In [56]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS indians_players_list (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    role VARCHAR(100),
    batting_style VARCHAR(100),
    bowling_style VARCHAR(100)
    )
""")
conn.commit()
print("Table created successfully")    

Table created successfully


In [57]:
count = 0
for player in indian_players:
    try:
        id = int(player['id'])
        name = player['name']
        role = player['role']
        batting_style = player['batting_style']
        bowling_style = player['bowling_style']
        cur.execute("""
            INSERT INTO indians_players_list (id, name, role, batting_style, bowling_style)
            VALUES (%s, %s, %s, %s, %s)
            ON CONFLICT (id) DO NOTHING        
        """, (
            id,
            name,
            role,
            batting_style,
            bowling_style
        ))
        count += 1
    except Exception as e:
        print(f"Error inserting player {player}: {e}")
conn.commit()
print(f"{count} records inserted successfully")
cur.close()
conn.close()

37 records inserted successfully


# Question 2

In [46]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"

headers = {
	"x-rapidapi-key": "5d00263db8msh6b461e04fff0e6bp1aa84cjsn57ff489cc9c5",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())
recent_matches = response.json()
print(recent_matches)

{'typeMatches': [{'matchType': 'International', 'seriesMatches': [{'seriesAdWrapper': {'seriesId': 11253, 'seriesName': "ICC Men's T20 World Cup 2026", 'matches': [{'matchInfo': {'matchId': 139227, 'seriesId': 11253, 'seriesName': "ICC Men's T20 World Cup 2026", 'matchDesc': '28th Match, Group D', 'matchFormat': 'T20', 'startDate': '1771219800000', 'endDate': '1771232400000', 'state': 'Complete', 'status': 'Afghanistan won by 5 wkts', 'team1': {'teamId': 7, 'teamName': 'United Arab Emirates', 'teamSName': 'UAE', 'imageId': 776242}, 'team2': {'teamId': 96, 'teamName': 'Afghanistan', 'teamSName': 'AFG', 'imageId': 776177}, 'venueInfo': {'id': 51, 'ground': 'Arun Jaitley Stadium', 'city': 'Delhi', 'timezone': '+05:30', 'latitude': '28.637891', 'longitude': '77.243073'}, 'currBatTeamId': 96, 'seriesStartDt': '1770422400000', 'seriesEndDt': '1773100800000', 'isTimeAnnounced': True, 'stateTitle': 'Complete'}, 'matchScore': {'team1Score': {'inngs1': {'inningsId': 1, 'runs': 160, 'wickets': 9,

In [47]:
type_matches = recent_matches['typeMatches']
series_matches = type_matches[0]['seriesMatches'][0]
series_wrapper = series_matches['seriesAdWrapper']
match_list = series_wrapper['matches']
first_match = match_list[0]
print(first_match['matchInfo'])

{'matchId': 139227, 'seriesId': 11253, 'seriesName': "ICC Men's T20 World Cup 2026", 'matchDesc': '28th Match, Group D', 'matchFormat': 'T20', 'startDate': '1771219800000', 'endDate': '1771232400000', 'state': 'Complete', 'status': 'Afghanistan won by 5 wkts', 'team1': {'teamId': 7, 'teamName': 'United Arab Emirates', 'teamSName': 'UAE', 'imageId': 776242}, 'team2': {'teamId': 96, 'teamName': 'Afghanistan', 'teamSName': 'AFG', 'imageId': 776177}, 'venueInfo': {'id': 51, 'ground': 'Arun Jaitley Stadium', 'city': 'Delhi', 'timezone': '+05:30', 'latitude': '28.637891', 'longitude': '77.243073'}, 'currBatTeamId': 96, 'seriesStartDt': '1770422400000', 'seriesEndDt': '1773100800000', 'isTimeAnnounced': True, 'stateTitle': 'Complete'}


In [48]:
matches = []

for type_match in recent_matches.get('typeMatches', []):

    for series_match in type_match.get('seriesMatches', []):
        series_wrapper = series_match.get('seriesAdWrapper', {})
        series_id = series_wrapper.get('seriesId')
        series_name = series_wrapper.get('seriesName')

        for match in series_wrapper.get('matches', []):
            info = match.get('matchInfo', {})
            match_id = info.get('matchId')
            match_desc = info.get('matchDesc')
            match_format = info.get('matchFormat')
            start_date = info.get('startDate')
            status = info.get('status')
            team1 = info.get('team1', {}).get('teamName')
            team2 = info.get('team2', {}).get('teamName')
            venue = info.get('venueInfo', {}).get('ground')

            matches.append({
                'series_id': series_id,
                'series_name': series_name,
                'match_id': match_id,
                'match_desc': match_desc,
                'match_format': match_format,
                'start_date': start_date,
                'status': status,
                'team1': team1,
                'team2': team2,
                'venue': venue
            })

In [53]:
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

Connected to the database successfully


In [50]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS recents_matches (
    series_id INTEGER,
    series_name VARCHAR(255),
    match_id INTEGER,
    match_desc TEXT,
    match_format VARCHAR(100),
    start_date DATE,
    status VARCHAR(100),
    team1 VARCHAR(255),
    team2 VARCHAR(255),
    venue VARCHAR(255)
    )
""")
conn.commit()
print("Table created successfully")  

Table created successfully


In [54]:
from datetime import datetime

count = 0
errors = []

for data in matches:
    try:
        series_id = data['series_id']
        series_name = data['series_name']
        match_id = data['match_id']
        match_desc = data['match_desc']
        match_format = data['match_format']
        start_date = data['start_date']
        status = data['status']
        team1 = data['team1']
        team2 = data['team2']
        venue = data['venue']
        
        
        if start_date:
            
            start_date = datetime.fromtimestamp(int(start_date) / 1000).date()
        else:
            start_date = None
        
        cur.execute("""
            INSERT INTO recents_matches (series_id, series_name, match_id, match_desc, match_format, start_date, status, team1, team2, venue)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (series_id, series_name, match_id, match_desc, match_format, start_date, status, team1, team2, venue))
        count += 1
    except Exception as e:
        errors.append(str(e))

conn.commit()
print(f"{count} records inserted successfully")
if errors:
    print(f"\nUnique errors encountered:")
    for error in set(errors[:5]): 
        print(f"  - {error}")
        
cur.close()
conn.close()

59 records inserted successfully


# Question 3

In [24]:
url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats/0"

querystring = {"statsType":"mostRuns","matchType":"2"}

headers = {
	"x-rapidapi-key": "5d00263db8msh6b461e04fff0e6bp1aa84cjsn57ff489cc9c5",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())
top_odi_runs = response.json()
top_odi = top_odi_runs['values']
player_ids = [int(item['values'][0]) for item in top_odi]

{'filter': {'matchtype': [{'matchTypeId': '1', 'matchTypeDesc': 'test'}, {'matchTypeId': '2', 'matchTypeDesc': 'odi'}, {'matchTypeId': '3', 'matchTypeDesc': 't20i'}], 'team': [{'id': '2', 'teamShortName': 'IND'}, {'id': '27', 'teamShortName': 'IRE'}, {'id': '3', 'teamShortName': 'PAK'}, {'id': '4', 'teamShortName': 'AUS'}, {'id': '5', 'teamShortName': 'SL'}, {'id': '6', 'teamShortName': 'BAN'}, {'id': '9', 'teamShortName': 'ENG'}, {'id': '10', 'teamShortName': 'WI'}, {'id': '11', 'teamShortName': 'RSA'}, {'id': '12', 'teamShortName': 'ZIM'}, {'id': '13', 'teamShortName': 'NZ'}, {'id': '96', 'teamShortName': 'AFG'}], 'selectedMatchType': 'odi'}, 'headers': ['Batter', 'M', 'I', 'R', 'Avg'], 'values': [{'values': ['25', 'Tendulkar', '463', '452', '18426', '44.83']}, {'values': ['1413', 'Virat Kohli', '311', '299', '14797', '58.72']}, {'values': ['104', 'Sangakkara', '404', '380', '14234', '41.99']}, {'values': ['38', 'R Ponting', '375', '365', '13704', '42.04']}, {'values': ['102', 'S Jay

In [25]:
player_data = {}

for vid in player_ids:
    try:
        url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{vid}/batting"
    
        response = requests.get(url, headers=headers, timeout=5)

        print(f"\n===== Series ID {vid} =====")

        if response.status_code == 200:
            player_data[vid] = response.json()
            print(player_data[vid])
        else:
            print("Failed:", response.status_code)

    except Exception as e:
        print("Error for", vid, e)

odi_hundreds = {}

for pid, pdata in player_data.items():
    for row in pdata['values']:
        if row['values'][0] == '100s':
            odi_hundreds[pid] = int(row['values'][2])
            break
odi_hundreds        


===== Series ID 25 =====
{'headers': ['ROWHEADER', 'Test', 'ODI', 'T20', 'IPL'], 'values': [{'values': ['Matches', '200', '463', '1', '78']}, {'values': ['Innings', '329', '452', '1', '78']}, {'values': ['Runs', '15921', '18426', '10', '2334']}, {'values': ['Balls', '29437', '21367', '12', '1948']}, {'values': ['Highest', '248', '200', '10', '100']}, {'values': ['Average', '53.79', '44.83', '10', '33.83']}, {'values': ['SR', '54.09', '86.24', '83.34', '119.82']}, {'values': ['Not Out', '33', '41', '0', '9']}, {'values': ['Fours', '2058', '2016', '2', '295']}, {'values': ['Sixes', '69', '195', '0', '29']}, {'values': ['Ducks', '14', '20', '0', '4']}, {'values': ['50s', '68', '96', '0', '13']}, {'values': ['100s', '51', '49', '0', '1']}, {'values': ['200s', '6', '1', '0', '0']}, {'values': ['300s', '0', '0', '0', '0']}, {'values': ['400s', '0', '0', '0', '0']}], 'appIndex': {'seoTitle': 'Sachin Tendulkar Profile - Cricbuzz | Cricbuzz.com', 'webURL': 'http://www.cricbuzz.com/profiles/25/

{25: 49,
 1413: 54,
 104: 25,
 38: 30,
 102: 28,
 101: 19,
 35: 10,
 213: 17,
 576: 33,
 29: 22,
 27: 12,
 265: 10,
 247: 25,
 240: 19,
 105: 22,
 34: 15,
 36: 16,
 370: 25,
 3864: 7,
 3531: 11}

In [30]:
rows = []

for pid, pdata in player_data.items():
    name = pdata['appIndex']['seoTitle'].split(' Profile')[0]

    odi_runs = odi_avg = odi_100s = None

    for row in pdata['values']:
        label = row['values'][0]
        if label == 'Runs':
            odi_runs = int(row['values'][2])
        elif label == 'Average':
            odi_avg = float(row['values'][2])
        elif label == '100s':
            odi_100s = int(row['values'][2])

    rows.append({
        'Player Name': name,
        'Runs': odi_runs,
        'Average': odi_avg,
        'Centuries': odi_100s
    })

top_odi_10_df = pd.DataFrame(rows).sort_values('Runs', ascending=False).head(10)
top_odi_10_df

,Player Name,Runs,Average,Centuries
0,Sachin Tendulkar,18426,44.83,49
1,Virat Kohli,14797,58.72,54
2,Kumar Sangakkara,14234,41.99,25
3,Ricky Ponting,13704,42.04,30
4,Sanath Jayasuriya,13430,32.13,28
5,Mahela Jayawardene,12650,33.38,19
6,Inzamam-ul-Haq,11739,39.53,10
7,Jacques Kallis,11579,44.36,17
8,Rohit Sharma,11577,48.85,33
9,Sourav Ganguly,11363,40.73,22


In [41]:
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
print("Connected to the database successfully")

Connected to the database successfully


In [42]:
# Create table
cur.execute("""
CREATE TABLE IF NOT EXISTS "Top 10 odi scorers" (
    player_name VARCHAR(100),
    runs INTEGER,
    average NUMERIC(5,2),
    centuries INTEGER
)
""")

# Insert query
insert_query = """
INSERT INTO "Top 10 odi scorers"
(player_name, runs, average, centuries)
VALUES (%s, %s, %s, %s)
"""

# Convert DataFrame → tuples
data = list(top_odi_10_df.itertuples(index=False, name=None))

# Insert
cur.executemany(insert_query, data)

conn.commit()
cur.close()


# Question 4

In [ ]:
url = "https://cricbuzz-cricket.p.rapidapi.com/venues/v1/11"

headers = {
	"x-rapidapi-key": "a017727962mshce1fb9c4eb63942p1275e2jsnfcdf53b20a76",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())